In [40]:
import nltk
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [41]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [42]:
en_stop = set(nltk.corpus.stopwords.words('english'))

In [43]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [44]:
import random
text_data = []
with open('../data/test4.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        text_data.append(tokens)
    

In [45]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [46]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [47]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [48]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [49]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"relationship" + 0.021*"global" + 0.021*"level" + 0.020*"model"')
(1, '0.079*"discourse" + 0.061*"intention" + 0.041*"segment" + 0.041*"among"')
(2, '0.053*"level" + 0.052*"segment" + 0.041*"discourse" + 0.021*"model"')
(3, '0.096*"structure" + 0.046*"discourse" + 0.039*"linguistic" + 0.030*"intentional"')
(4, '0.089*"topic" + 0.071*"document" + 0.040*"threads" + 0.033*"connect"')


In [50]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.029*"intentional" + 0.028*"threads" + 0.019*"intention" + 0.019*"segment"')
(1, '0.095*"discourse" + 0.049*"structure" + 0.038*"segment" + 0.035*"level"')
(2, '0.087*"topic" + 0.056*"document" + 0.030*"words" + 0.027*"connect"')


In [51]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.089*"structure" + 0.060*"linguistic" + 0.032*"intention" + 0.032*"relationship"')
(1, '0.154*"topic" + 0.096*"document" + 0.048*"words" + 0.048*"connect"')
(2, '0.073*"segment" + 0.038*"intention" + 0.038*"relationship" + 0.038*"intentional"')
(3, '0.064*"intention" + 0.064*"tutorial" + 0.034*"provide" + 0.034*"among"')
(4, '0.064*"model" + 0.064*"grosz" + 0.034*"level" + 0.034*"relationship"')
(5, '0.035*"utterance" + 0.035*"theory" + 0.035*"within" + 0.035*"grice"')
(6, '0.110*"segment" + 0.108*"discourse" + 0.048*"coherence" + 0.047*"purpose"')
(7, '0.097*"structure" + 0.097*"discourse" + 0.056*"state" + 0.056*"attentional"')
(8, '0.046*"document" + 0.046*"threads" + 0.023*"solve" + 0.023*"connect"')
(9, '0.076*"discourse" + 0.039*"sidner" + 0.039*"among" + 0.039*"relationship"')


In [52]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))

In [53]:
import pyLDAvis.gensim

In [54]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/home/eesha/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [55]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/home/eesha/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
